In [1]:
import os

import matplotlib.pyplot as plt
import tensorflow as tf

from src.dataset import *
from src.model import *

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
# TFRWriter().write()

In [3]:
config = {
    'local_depth': 16,
    'local_convdepth': 32,
    'global_depth': 5,
    'global_convdepth': 64,
    'global_imagesize': 128,
    'scale': 0.25}

model = CIEXYZNet(**config)

2022-05-31 08:44:36.608832: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:938] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0d:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-31 08:44:36.636598: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:938] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0d:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-31 08:44:36.636996: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:938] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0d:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-31 08:44:36.637450: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with t

In [4]:
val_data = DataLoader().get_dataset('validation')

In [5]:
for i, val_batch in enumerate(val_data):
    logits = model(val_batch['sRGB_image'])
    print(i, logits)

2022-05-31 08:44:38.885929: I tensorflow/stream_executor/cuda/cuda_dnn.cc:379] Loaded cuDNN version 8400
2022-05-31 08:44:39.835195: I tensorflow/stream_executor/cuda/cuda_blas.cc:1804] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


0 (<tf.Tensor: shape=(8, 256, 256, 3), dtype=float32, numpy=
array([[[[ 1.44259575e+06, -8.49959400e+06, -4.75064000e+06],
         [ 1.30395762e+06, -7.95912950e+06, -4.43852400e+06],
         [ 1.50908625e+06, -8.92450100e+06, -4.97598400e+06],
         ...,
         [-2.41127047e+05, -2.53851850e+06, -1.24989912e+06],
         [-3.54616188e+05, -2.60715475e+06, -1.20049888e+06],
         [-4.15179031e+05, -2.50329675e+06, -1.11824562e+06]],

        [[ 1.39212800e+06, -8.91395700e+06, -4.87031550e+06],
         [ 1.37414675e+06, -8.94733000e+06, -4.87722400e+06],
         [ 1.37414675e+06, -8.94733000e+06, -4.87722400e+06],
         ...,
         [-3.35581281e+05, -2.68485300e+06, -1.24824525e+06],
         [-3.68999406e+05, -2.02516788e+06, -9.05494000e+05],
         [-3.74157219e+05, -2.22442325e+06, -1.00191400e+06]],

        [[ 1.51835012e+06, -9.51918600e+06, -5.20392000e+06],
         [ 1.45737100e+06, -9.34924000e+06, -5.09851950e+06],
         [ 1.47890975e+06, -9.38641900e

In [6]:
model.summary()

Model: "ciexyz_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sRGB2XYZ_localnet (Sequenti  (None, 256, 256, 3)      131235    
 al)                                                             
                                                                 
 XYZ2sRGB_localnet (Sequenti  (None, 256, 256, 3)      131235    
 al)                                                             
                                                                 
 sRGB2XYZ_globalnet (Sequent  (None, 18)               1217554   
 ial)                                                            
                                                                 
 XYZ2sRGB_globalnet (Sequent  (None, 18)               1217554   
 ial)                                                            
                                                                 
Total params: 2,697,578
Trainable params: 2,697,578
Non-

In [7]:
model.srgb2xyz_localnet.summary()

Model: "sRGB2XYZ_localnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_0 (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv_1 (Conv2D)             (None, 256, 256, 32)      9248      
                                                                 
 conv_2 (Conv2D)             (None, 256, 256, 32)      9248      
                                                                 
 conv_3 (Conv2D)             (None, 256, 256, 32)      9248      
                                                                 
 conv_4 (Conv2D)             (None, 256, 256, 32)      9248      
                                                                 
 conv_5 (Conv2D)             (None, 256, 256, 32)      9248      
                                                                 
 conv_6 (Conv2D)             (None, 256, 256, 32)

In [8]:
model.srgb2xyz_globalnet.summary()

Model: "sRGB2XYZ_globalnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 128, 128, 3)       0         
                                                                 
 conv_0 (Conv2D)             (None, 128, 128, 64)      1792      
                                                                 
 maxpool_0 (MaxPooling2D)    (None, 64, 64, 64)        0         
                                                                 
 conv_1 (Conv2D)             (None, 64, 64, 64)        36928     
                                                                 
 maxpool_1 (MaxPooling2D)    (None, 32, 32, 64)        0         
                                                                 
 conv_2 (Conv2D)             (None, 32, 32, 64)        36928     
                                                                 
 maxpool_2 (MaxPooling2D)    (None, 16, 16, 64) 